### Evaluating Retrieval Augmented Geneation(RAG) Using Ollama model.

- Note: In the course, OpenAI GPT was used

In [1]:
# Install Ollama into the notebook runtime

!curl -fsSL https://ollama.com/install.sh | sh

>>> Installing ollama to /usr/local
>>> Downloading Linux amd64 bundle
######################################################################## 100.0%
>>> Creating ollama user...
>>> Adding ollama user to video group...
>>> Adding current user to ollama group...
>>> Creating ollama systemd service...
>>> The Ollama API is now available at 127.0.0.1:11434.
>>> Install complete. Run "ollama" from the command line.


In [2]:
# Start Ollama using `ollama serve`. This needs to run in the backgound - so we run it using `nohup` (to see the output log, open nohup.out).

!nohup ollama serve > nohup.out 2>&1 &

* Pull desired Ollama Model (gemma3:1b)

In [3]:
%%capture
!ollama pull gemma3:1b

In [4]:
!pip install ollama

In [5]:
import ollama

In [6]:
! pip install evidently langchain langchain_community faiss-cpu sentence_transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 111.0/111.0 kB 9.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.2/5.2 MB 88.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 82.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.3/31.3 MB 61.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 236.5/236.5 kB 24.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 573.2/573.2 kB 47.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 92.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 56.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 46.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664

In [20]:
import random
import requests
import time
import os
import pandas as pd

from langchain.text_splitter import CharacterTextSplitter
from langchain.docstore.document import Document
from langchain.vectorstores import FAISS
from langchain.embeddings import SentenceTransformerEmbeddings

#from sentence_transformers import SentenceTransformer

In [8]:
from evidently.ui.workspace import CloudWorkspace

from evidently import Dataset, DataDefinition, Report
from evidently.descriptors import *
from evidently.presets import TextEvals
from evidently.llm.templates import BinaryClassificationPromptTemplate

## Simulate RAG system

In [21]:
def load_and_index_from(url):
    # Step 1: Load file content from GitHub raw URL
    response = requests.get(url)
    response.raise_for_status()  # Raise error if download fails
    content = response.text

    # Step 2: Split into chunks
    text_splitter = CharacterTextSplitter(chunk_size=800, chunk_overlap=200)
    documents = [Document(page_content=content)]
    chunks = text_splitter.split_documents(documents)

    # Step 3: Generate embeddings and create FAISS index
    embedding_model = SentenceTransformerEmbeddings(model_name="all-MiniLM-L6-v2")
    vector_store = FAISS.from_documents(chunks, embedding_model)

    return vector_store

In [17]:
# Search the index for relevant information
def search_documents(query, vector_store):
    search_results = vector_store.similarity_search(query, k=5)
    context = "\n".join([doc.page_content for doc in search_results])
    return context

In [18]:
instructions="Your task is to answer the provided question based on the context."

def generate_response(question, context, model):
  response = ollama.chat(
    model="gemma3:1b",
    messages=[
        {"role": "system", "content": instructions},
        {"role": "user", "content": f"The retrieved context is {context} \n {question}"}
    ]
)

  print(response['message']['content'])

### Testing our RAG System

In [22]:
url = "https://docs.evidentlyai.com/llms-full.txt"
vector_store = load_and_index_from(url)

<ipython-input-21-d0c3549b4b25>:13: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embedding_model = SentenceTransformerEmbeddings(model_name="all-MiniLM-L6-v2")


### Load Evidently Cloud

In [9]:
from google.colab import userdata

EVIDENTLY_ORG_ID = userdata.get('EVIDENTLY_ORG_ID')
EVIDENTLY_TOKEN = userdata.get('EVIDENTLY_TOKEN')

In [10]:
client = CloudWorkspace(url="https://app.evidently.cloud", token=EVIDENTLY_TOKEN)

Load testing dataset

In [14]:
dataset_id = "0196ed73-ca92-701a-bfd6-97118c6969f6"
testing_dataset = client.load_dataset(dataset_id, d)

TypeError: CloudWorkspace.load_dataset() got an unexpected keyword argument 'token'

- Note: I do not have access to the test dataset used here, so I can't continue with the use of Ollama models for RAG.

* Reference the LLMCourse_RAG_Evals notebook for more.